In [1]:
import pandas as pd
import numpy as np

#saving numpy state
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [2]:
#erading in top 3500 ingredients
with np.load('simplified-recipes-1M.npz') as data:
    recipes = data['recipes']
    ingredients = data['ingredients']
    
#reseting numpy 
np.load = np_load_old

In [3]:
ingredients = list(ingredients)
ingredients = [i for i in ingredients if len(i) > 2]
ingredients = [e for e in ingredients if e not in ('ingredients', 'fresh', 'love', 'sliced', 'black', 'plus', 'whole', 'dry', 'packed', 'more', 'temperature', 'sweetened', 'soft', 'other', 'unsweetened', 'raw', 'half', 'mix', 'flavor', 'meal', 'flavored')]

In [4]:
i_out = np.asarray(ingredients)
np.savetxt('ingredients.txt', i_out, fmt='%s', delimiter = ',')

In [5]:
full = pd.read_csv('epi_r.csv')
json = pd.read_json('full_format_recipes.json')

In [6]:
json = json.drop(columns=['categories'])
json[['directions', 'ingredients']] = json[['directions', 'ingredients']].astype(str)
full = full.drop_duplicates()
json = json.drop_duplicates()

In [7]:
#merging two dataframes to include ingredients and ratings
combined = full.merge(json, on=['title','rating','calories'], suffixes=('', '_y'))

#dropping categories
combined_clean = combined.drop(columns=combined.columns[6:-7])
combined_clean = combined_clean.drop(columns=['date_y', 'fat_y', 'protein_y', 'sodium_y'])

In [8]:
#grabbing the ingredients from the top list and checking if is in recipe list
combined_clean['ingredients'] = combined_clean['ingredients'].astype(str)

output1 = []
for ing in ingredients:
    items = combined_clean['ingredients']
    output2 = []
    for i in items:
        output2.append(ing in i)
    output1.append(output2)

output_dict = {}
for o, ing in zip(output1, ingredients):
    output_dict[ing] = o

In [9]:
#adding new top ingredients to dataframe and saving to new csv
ing_df = pd.DataFrame(output_dict)
full_df = pd.concat([combined_clean, ing_df], axis=1)

In [10]:
ing_vals = full_df.iloc[:, 10:].sum(axis=0)
ing_vals = ing_vals.loc[ing_vals <= 5].index
full_df = full_df.drop(columns = list(ing_vals))
full_df = full_df.sort_values('rating', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
full_df = full_df.drop_duplicates()
full_df = full_df.iloc[:len(full_df)//8]
full_df.to_csv('full_df.csv')

# Recommender 

In [ ]:
#test case
food_items = ['wine', 'honey', 'fish']

output = np.zeros(len(full_df))
for food in food_items:
    output += full_df[food]
    
food_sim = pd.DataFrame({'similarity':output})
top = list(food_sim.sort_values(by=['similarity'], ascending=False).index)